# Modelling Data

In [3]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
import geopandas as gpd
import h3
from shapely import wkt
from sklearn.linear_model import LinearRegression
import seaborn as sns
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import warnings
import requests
import datetime
from bs4 import BeautifulSoup
warnings.filterwarnings('ignore')

## Creating our panel 

### Read in data 

In [9]:
data_path = r'C:\Users\noah2\OneDrive\Desktop\GSE 580\Final Model\Github Corrections\waze-romania\data'

waze = pd.read_csv(data_path + '/fullclean.csv')
waze.drop(columns=['confidence', 'nThumbsUp', 'country'])
waze['date']= pd.to_datetime(waze['date'])

schools = pd.read_csv(data_path + '/schools.csv')


hosp = pd.read_csv(data_path + '/hospitals.csv')

#Weather data is from https://www.ecad.eu/
weather = pd.read_csv(data_path + '/RainLevels.csv')[['Date', 'Precipitation']]
weather.columns = ['date', 'precip']
weather['date']= pd.to_datetime(weather['date'])


#Stringency index is from https://ourworldindata.org/metrics-explained-covid19-stringency-index
covid = pd.read_csv(data_path + '/stringency.csv')
covid = covid[['date', 'stringency_index']]
covid = covid[covid.date<='2022-01-01']  
covid['date']= pd.to_datetime(covid['date'])

#### Webscraping the soccer and holidays data

##### Holidays

In [10]:
years = [2020, 2021, 2022]
holidays = []
for year in years:
    URL = 'https://leaveboard.com/public-holidays/romania-public-holidays-'+ str(year) + '/'
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")
    rawholidays = soup.find_all("div", class_ = 'd-md-none')
    for holiday in rawholidays:
        temp = holiday.text[4:]
        temp = datetime.datetime.strptime(temp, '%B %d, %Y')
        holidays.append(temp)
holidays = pd.DataFrame(holidays)
holidays.columns= ['date']
holidays['IsHoliday'] = 1

##### Soccer (for both teams in Cluj)

In [11]:
cluj = []

for year in years:
    URL = 'https://www.espn.com/soccer/team/results/_/id/5260/season/' + str(year)
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")
    tables = soup.find_all(class_ = "ResponsiveTable")
    for table in tables:
        datesraw = table.find_all(class_ = 'matchTeams')
        gamesraw = table.find_all(class_ = 'local flex items-center')
        year  = table.find_all(class_ = 'Table__Title')[0].text[-4:]
        for i in range(len(gamesraw)):
            if gamesraw[i].text == 'CFR Cluj-Napoca':
                temp = datesraw[i].text[5:] +', ' + year
                temp = datetime.datetime.strptime(temp, '%b %d, %Y')
                cluj.append(temp)
univ = []
for year in years:
    URL = 'https://www.espn.com/soccer/team/results/_/id/8089/season/' + str(year)
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")
    tables = soup.find_all(class_ = "ResponsiveTable")
    for table in tables:
        datesraw = table.find_all(class_ = 'matchTeams')
        gamesraw = table.find_all(class_ = 'local flex items-center')
        year  = table.find_all(class_ = 'Table__Title')[0].text[-4:]
        for i in range(len(gamesraw)):
            if gamesraw[i].text == 'Universitatea Craiova':
                temp = datesraw[i].text[5:] +', ' + year
                temp = datetime.datetime.strptime(temp, '%b %d, %Y')
                univ.append(temp)

cluj = pd.DataFrame(cluj)
cluj.columns= ['date']
cluj['CFRGame'] = 1

univ = pd.DataFrame(univ)
univ.columns= ['date']
univ['UnivGame'] = 1

soccer = pd.merge(cluj, univ, how = 'outer')
soccer = soccer.fillna(0)

#### Reading and formatting bus stop data

In [14]:
busstops = pd.read_csv(data_path + '/busstops.csv')
for i in [6,7,8,9,10]:
    busstops["h" + str(i)] = busstops.apply(lambda x: h3.geo_to_h3(x["lat"], x["lon"], i), axis=1)
busstops = busstops.drop(['Unnamed: 0', 'Stop', 'City', 'lat', 'lon'], axis = 1)
busstops['BusStops'] = 1

### Merging the Data
Caveat: we need to think about how the different hexagon resolutions are impacting this merge. Are there any potential issues being introduced?

In [15]:
# merging stringency and precipitation data by date
df = waze.merge(covid,how='left', on='date')
df = df.merge(weather, how='left', on='date')
df = df.merge(soccer, how='left', on='date')
df = df.merge(holidays, how='left', on='date')

# merging school and hospital data by hexagon
df = df.merge(schools, how='left', on=['h6', 'h7', 'h8', 'h9', 'h10'])
df = df.merge(hosp, how='left', on=['h6', 'h7', 'h8', 'h9', 'h10'])
df = df.merge(busstops, how='left', on=['h6', 'h7', 'h8', 'h9', 'h10'])

df['UnivGame'] = df['UnivGame'].fillna(0)
df['CFRGame'] = df['CFRGame'].fillna(0)
df['IsHoliday'] = df['IsHoliday'].fillna(0)

col_dict = {"Denumire_P": "schools", "Nume": "hospitals", "uuid": "alerts", "stringency_index": "stringency"}

print('The final dataframes will look like...')
dfs = []
for i in [9, 10]:
    cols = ['date','h6', 'h7','h' + str(i) ,'stringency_index', 'precip', 'dayofweek', 'month', 'CFRGame', 'UnivGame', 'IsHoliday']
    x = df.groupby(cols,as_index=False)[['uuid', 'Denumire_P', 'Nume', 'BusStops']].count()
    x = x.rename(columns = col_dict)
    dfs.append(x)
dfs[1]

The final dataframes will look like...


,date,h6,h7,h10,stringency,precip,dayofweek,month,CFRGame,UnivGame,IsHoliday,alerts,schools,hospitals,BusStops
0,2020-02-26,861e0b217ffffff,871e0b210ffffff,8a1e0b210d37fff,16.67,5,2,2,0.0,0.0,0.0,1,0,0,0
1,2020-02-26,861e0b217ffffff,871e0b212ffffff,8a1e0b21282ffff,16.67,5,2,2,0.0,0.0,0.0,1,0,0,0
2,2020-02-26,861e0b217ffffff,871e0b214ffffff,8a1e0b214537fff,16.67,5,2,2,0.0,0.0,0.0,2,0,0,0
3,2020-02-26,861e0b217ffffff,871e0b214ffffff,8a1e0b216b4ffff,16.67,5,2,2,0.0,0.0,0.0,1,0,0,0
4,2020-02-26,861e0b217ffffff,871e0b214ffffff,8a1e0b38926ffff,16.67,5,2,2,0.0,0.0,0.0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199278,2021-12-31,861e0b3afffffff,871e0b3acffffff,8a1e0b3ac54ffff,52.78,8,4,12,0.0,0.0,0.0,2,0,0,0
199279,2021-12-31,861e0b3afffffff,871e0b3aeffffff,8a1e0b3ae66ffff,52.78,8,4,12,0.0,0.0,0.0,1,0,0,0
199280,2021-12-31,861e0b76fffffff,871e0b769ffffff,8a1e0b769a77fff,52.78,8,4,12,0.0,0.0,0.0,2,0,0,0
199281,2022-01-01,861e0b3afffffff,871e0b3acffffff,8a1e0b3ac187fff,52.78,1,5,1,0.0,0.0,1.0,1,0,0,0


### Create interactions

1. get all polynomial terms for numerical variables
2. add these to controls
3. interact all variables

In [16]:
%%time
panels = []
loop=1
for x in dfs:
    controls = x.iloc[:,4:]
    controls = controls.loc[:, controls.columns != 'alerts']
    num = controls[['stringency', 'precip', 'schools', 'hospitals', 'BusStops']]
    cat = controls[['dayofweek', 'month', 'UnivGame', 'CFRGame', 'IsHoliday', ]]

    # get polynomial terms for numerical variables
    for k in range(len(num.columns)):
        for p in range(3):
            if (p)>0:
                num[str(num.columns[k]) + '^' + str(p+1)] = num[num.columns[k]]**(p+1)
    controls = pd.concat((num, cat), axis=1) # controls including polynomial terms
    print('df ' + str(loop) + ' polynomials are ready')

    # now interact polynomial terms with categorical variables
    interaction = PolynomialFeatures(degree=3, include_bias=False, interaction_only=True)
    controls = pd.DataFrame(interaction.fit_transform(controls),
                                columns=interaction.get_feature_names(input_features=controls.columns))
    panels.append(pd.concat((x.iloc[:,:4],x[['alerts']], controls), axis=1))
    print('df ' + str(loop) + ' is interacted')
    loop+=1


d = dict(zip([9,10], panels)) 
for key in d:
    d[key].to_csv(data_path + '/model' + str(key) + '.csv', index = False)
    print('resolution' + str(key) + 'panel is written to disk')

df 1 polynomials are ready
df 1 is interacted
df 2 polynomials are ready
df 2 is interacted
resolution9panel is written to disk
resolution10panel is written to disk
CPU times: total: 3min 43s
Wall time: 4min 24s
